In [1]:
# March 22, 2019 9:40PM
# alldata is from Aug 28 to March 20

# DATASET6
# a is from 3/15/19 10:45AM-12:15PM
# b is from 3/15/19 1:30-2:35
# c is from 3/15/19 2:40-3:23

# DATASET 7
# fullrow2 from 3/20/19 11:00-12:00
# fullrow3 from 3/20/19 12:30-13:30 -- the sensors went down or something... 
# atrain2 from 3/20/19 14:00-15:00
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.interpolate import interp1d
import os

os.chdir('/Users/Aldis/Documents/MATLAB/ES100/Dataset7/ALLDATA2')

# Import and rename columns

In [2]:
# from filedrop into new folder dataset4, import directly
# read_csv with delimiter as a semicolon, rename headers, parse dates as indices! 
# aq4 = pd.read_csv('aq4.csv', delimiter = ';', parse_dates = [0], index_col = [0], header = 0, names = ['occ', 'co2','qfresh','humid'])

def rename(file_csv, names_array):
    ''' uses pd.read_csv to quickly read and rename new imported datasets
        give the name of the file (e.g. 'aq4.csv', and the names_array = ['occ', 'co2',...])
        returns dataset with an abbreviation of your choice (output)'''
    
    dataset = pd.read_csv(file_csv, delimiter = ';',
                         parse_dates = [0], index_col = [0],
                         header = 0, names = names_array) # 
    
    # convert to dataframe
    dataseries = pd.DataFrame(dataset, dtype = float) # to be able to resample later: https://stackoverflow.com/questions/12844529/no-numeric-types-to-aggregate-change-in-groupby-behaviour

    return dataseries

def rename_logger(file_csv, names_array):
    ''' uses pd.read_csv to quickly read and rename new imported datasets
        give the name of the file (e.g. 'aq4.csv', and the names_array = ['occ', 'co2',...])
        returns dataset with an abbreviation of your choice (output)'''
    
    # read into a dataset
    dataset = pd.read_csv(file_csv, delimiter = ',', skiprows = 2, 
                      parse_dates = [1], index_col = [1], names = ['#','time', names_array]) 
    
    # convert to dataframe for processing
    dataframe = pd.DataFrame(dataset) # to be able to resample later: https://stackoverflow.com/questions/12844529/no-numeric-types-to-aggregate-change-in-groupby-behaviour
    
    # remove HOBOware indexing column
    dataseries = dataframe.drop(['#'],1)
    
    return dataseries

In [3]:
# convert fahrenheit values to C and replace that (first) column of the dataset
def convert2C(singlestream, replace_col):
    ''' F to C conversion '''
    arrayC = np.zeros(len(singlestream.values))
    for i in np.arange(0, len(singlestream.values),1):
        arrayC[i] = (5/9)*(singlestream.values[i].astype('float')-32)
    arrayC # not good, it rounds
    singlestream.iloc[:,replace_col] = np.transpose(arrayC)
    return singlestream

# Single data streams with ºF to ºC conversion

In [4]:
# importing them separately allows us to interpolate immediately
# but requires more concatenation later
os.chdir('/Users/Aldis/Documents/MATLAB/ES100/Dataset7/ALLDATA2')

positR = rename('positR.csv', ['positR'])
positS = rename('positS.csv', ['positS'])
heatvalve = rename('heatvalve.csv',['heatvalve'])

co2 = rename('co2.csv',['co2'])
occ = rename('occ.csv', ['occ'])
humid = rename('humid.csv', ['humid'])

# wasn't able to retrieve before internet explorer stopped responding
rain = rename('rain.csv', ['rain'])
windir = rename('windir.csv', ['windir'])
windsp = rename('windsp.csv',['windsp'])

oatC = rename('oatF.csv', ['oatC']) # pre-emptively renaming to oatC
oatC = convert2C(oatC, 0) # replace column 0

roomC = rename('roomC.csv', ['roomC'])
slabC = rename('slabC.csv', ['slabC'])
setC = rename('setF.csv', ['setC'])
setC = convert2C(setC, 0)
temp22 = rename('temp22.csv',['temp22'])
temp31 = rename('temp31.csv',['temp31'])
temp33 = rename('temp33.csv',['temp33'])

# find zeros in setpoint and replace with NaN
zeros = np.where(setC == 0) 
setC.iloc[zeros] = np.NaN

# display all
display(positR.head(), positS.head(),
        heatvalve.head(), co2.head(),
        rain.head(), windir.head(), windsp.head(),
        occ.head(), humid.head(),
        oatC.head(), roomC.head(),
        slabC.head(), setC.head(),
        temp22.head(), temp33.head())

,positR
2018-07-21 03:53:47,0.0
2018-08-28 10:08:29,2.0
2018-08-28 10:08:30,3.0
2018-08-28 10:08:31,4.0
2018-08-28 10:08:32,5.0


,positS
2018-07-21 03:53:38,0.0
2018-08-28 09:31:38,1.0
2018-08-28 09:31:39,4.0
2018-08-28 09:31:40,6.0
2018-08-28 09:31:41,9.0


,heatvalve
2018-08-27 09:57:44,100.0
2018-08-28 11:15:42,100.0
2018-08-29 08:45:15,100.0
2018-08-30 10:58:29,0.0
2018-08-30 10:59:03,100.0


,co2
2018-08-27 18:22:31,668.799988
2018-08-28 09:12:24,734.719971
2018-08-28 09:12:34,790.400024
2018-08-28 09:12:46,852.479980
2018-08-28 09:18:22,934.400024


,rain
2018-08-27 10:00:15,-1.0
2018-08-28 09:29:04,0.0
2018-08-28 09:45:31,-1.0
2018-08-28 09:50:31,0.0
2018-08-28 09:50:31,-1.0


...

,roomC
2018-08-27 23:14:58,27.400000
2018-08-28 00:59:28,27.180000
2018-08-28 02:25:24,26.959999
2018-08-28 03:48:16,26.680000
2018-08-28 07:11:16,26.480000


,slabC
2018-08-27 17:13:33,26.9
2018-08-28 01:12:43,26.6
2018-08-28 02:52:33,26.4
2018-08-28 05:42:15,26.1
2018-08-28 10:13:47,26.4


,setC
2018-08-27 09:58:05,22.0
2018-08-28 11:15:01,NaN
2018-08-28 11:15:42,22.0
2018-08-29 08:45:14,22.0
2018-08-30 10:59:01,21.0


,temp22
2018-08-27 23:35:29,26.080000
2018-08-28 00:13:27,25.860001
2018-08-28 01:15:13,25.600000
2018-08-28 02:51:53,25.320000
2018-08-28 03:49:05,25.020000


,temp33
2018-08-27 23:58:27,26.260000
2018-08-28 01:08:29,26.059999
2018-08-28 02:31:34,25.840000
2018-08-28 03:43:51,25.639999
2018-08-28 05:36:56,25.379999


# Handling data gaps, resampling, interpolating

In [5]:
# INTERPOLATE! -11:40PM
# built on interpolation function from stackoverflow:
# https://stackoverflow.com/questions/6518811/interpolate-nan-values-in-a-numpy-array
def interpolate(padata, pkind = 'slinear'): # spline 1st order
    '''linear interpolation
        takes in data with NaNs, and returns data with interpolated values'''
    
    aindexes = np.arange(padata.shape[0])
    
    ''' adding a for loop to do this for all columns of the data'''

    interpolated = np.zeros((padata.shape[0],padata.shape[1])) # rows, columns
    copy = padata # copy format of data table
    
    for i in np.arange(padata.shape[1]):
        agood_indexes, = np.where(np.isfinite(padata.iloc[:,i])) # non NaN values
        f = interp1d(agood_indexes
                    , padata.iloc[agood_indexes,i]
                    , bounds_error = False
                    , copy = False
                    , fill_value = "extrapolate"
                    , kind = pkind)
        interpolated[:,i] = f(aindexes)
        copy.iloc[:,i] = interpolated[:,i] # replace columns with interpolated data! IT WORKS!
        
    return copy

# handling data gaps either by filling with 0s or interpolating (continuous datasets)
def takemeans(dataset, Ts):
    '''resample and fillna(0)'''
    datasetm = dataset.resample(Ts).mean()
    return datasetm

def cleanfill(dataset, Ts):
    '''resample and fillna(0)'''
    dataseti = dataset.resample(Ts).mean().fillna(0)
    return dataseti

def cleanfwdfill(dataset, Ts):
    '''resample and fillna(0)'''
    dataseti = dataset.resample(Ts).mean().ffill()
    return dataseti
    
def cleaninterp(dataset, Ts):
    '''resample and interpolate
        intended for continuous datastreams like temp, humidity'''
    dataseti = interpolate(dataset.resample(Ts).mean())
    return dataseti

# NOW INTERPOLATE TO FILL NAs FORWARD

In [11]:
Ts = '5S' # 1 minute

## fill forward because settings are that if it doesn't change, no notes taken...
setCi = cleanfwdfill(setC,Ts)
positRi = cleanfwdfill(positR,Ts)
positSi = cleanfwdfill(positS,Ts)
heatvalvei = cleanfwdfill(heatvalve,Ts)
occi = cleanfwdfill(occ,Ts)
raini = cleanfwdfill(rain,Ts)


# # mostly interpolated
co2i = cleaninterp(co2,Ts)
humidi = cleaninterp(humid,Ts)
oatCi = cleaninterp(oatC,Ts)
temp22i = cleaninterp(temp22,Ts)
temp31i = cleaninterp(temp31,Ts)
temp33i = cleaninterp(temp33,Ts)
roomCi = cleaninterp(roomC,Ts)
slabCi = cleaninterp(slabC,Ts)
windiri = cleaninterp(windir,Ts)
windspi = cleaninterp(windsp,Ts)

# Now concatenate this

In [12]:
coni = pd.concat([positRi, positSi, heatvalvei, slabCi, humidi, co2i, 
                  occi, raini, windiri, windspi, oatCi, roomCi, setCi, 
                  temp33i, temp31i, temp22i], sort = True, axis=1) # setC is extra for tracking error calc
coni.head()

,positR,positS,heatvalve,slabC,humid,co2,occ,rain,windir,windsp,oatC,roomC,setC,temp33,temp31,temp22
2018-07-21 03:53:35,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-21 03:53:40,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-21 03:53:45,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-21 03:53:50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-21 03:53:55,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
full_rowi = coni[coni.iloc[:,0].notnull() 
               & coni.iloc[:,1].notnull() & coni.iloc[:,2].notnull()
               & coni.iloc[:,3].notnull() & coni.iloc[:,4].notnull() 
               & coni.iloc[:,5].notnull() & coni.iloc[:,6].notnull() 
               & coni.iloc[:,7].notnull() & coni.iloc[:,8].notnull() 
               & coni.iloc[:,9].notnull() & coni.iloc[:,10].notnull() 
               & coni.iloc[:,11].notnull() & coni.iloc[:,12].notnull() 
               & coni.iloc[:,13].notnull() & coni.iloc[:,14].notnull() 
               & coni.iloc[:,15].notnull()] # the not null command chopped off beginning and end (whatever hadn't interpolated between)

os.chdir('/Users/Aldis/Documents/MATLAB/ES100/Dataset7/')
full_rowi.to_csv('Preprocessed/032219/alldata-interp5sec.csv', index = True, header = True)
full_rowi.head() # higher resolution for control trial

,positR,positS,heatvalve,slabC,humid,co2,occ,rain,windir,windsp,oatC,roomC,setC,temp33,temp31,temp22
2018-08-28 00:00:30,0.0,0.0,100.0,26.645183,61.902516,693.839457,0.0,-1.0,293.473684,0.900,25.977241,27.304035,22.0,26.254048,26.026496,25.934781
2018-08-28 00:00:35,0.0,0.0,100.0,26.645131,61.902836,693.845630,0.0,-1.0,293.578947,0.925,25.976896,27.303860,22.0,26.253810,26.026329,25.934299
2018-08-28 00:00:40,0.0,0.0,100.0,26.645079,61.903157,693.851804,0.0,-1.0,293.684211,0.950,25.976552,27.303684,22.0,26.253572,26.026162,25.933816
2018-08-28 00:00:45,0.0,0.0,100.0,26.645026,61.903477,693.857977,0.0,-1.0,293.789474,0.975,25.976207,27.303509,22.0,26.253334,26.025996,25.933334
2018-08-28 00:00:50,0.0,0.0,100.0,26.644974,61.903798,693.864151,0.0,-1.0,293.894737,1.000,25.975862,27.303333,22.0,26.253095,26.025829,25.932851


In [14]:
full_rowi.tail()

,positR,positS,heatvalve,slabC,humid,co2,occ,rain,windir,windsp,oatC,roomC,setC,temp33,temp31,temp22
2019-03-20 15:05:25,0.0,4.4,12.0,20.6,21.76,544.960022,0.0,0.0,136.0,2.7,12.38,17.879999,21.677601,20.1,20.4,20.68
2019-03-20 15:05:30,0.0,4.4,12.0,20.6,21.76,544.960022,0.0,0.0,136.0,2.7,12.38,17.879999,21.677601,20.1,20.4,20.68
2019-03-20 15:05:35,0.0,4.4,12.0,20.6,21.76,544.960022,0.0,0.0,136.0,2.7,12.38,17.879999,21.677601,20.1,20.4,20.68
2019-03-20 15:05:40,0.0,4.4,12.0,20.6,21.76,544.960022,0.0,0.0,136.0,2.7,12.38,17.879999,21.677601,20.1,20.4,20.68
2019-03-20 15:05:45,0.0,0.0,12.0,20.6,21.76,544.960022,0.0,0.0,136.0,2.7,12.38,17.879999,21.677601,20.1,20.4,20.68
